In [7]:
from selenium import webdriver
driver = webdriver.Chrome()

In [8]:
# # 다음 페이지로
# driver.find_element(By.CSS_SELECTOR, "#paging > a.spr_com.page-next").click()



In [9]:
# # 이전 페이지로
# driver.find_element(By.CSS_SELECTOR, "#paging > a.spr_com.page-prev").click()


관광지

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from tqdm import tqdm
import time

# Chrome WebDriver 사용 (다운로드 링크: https://sites.google.com/chromium.org/driver/)
# driver_path = '/path/to/chromedriver'
driver = webdriver.Chrome()

url = "https://visitjeju.net/kr/detail/list?menuId=DOM_000001718000000000&cate1cd=cate0000000002#p1&pageSize=1490&sortListType=reviewcnt&viewType=map&isShowBtag&tag"
driver.get(url)

# 페이지 로드를 기다립니다. 여기서는 간단히 10초 대기하도록 설정하였습니다.
driver.implicitly_wait(10)

# 필요한 요소 추출
s_tit_elements = driver.find_elements(By.CLASS_NAME, 's_tit')
s_theme_elements = driver.find_elements(By.CLASS_NAME, 's_theme')
item_tag_next_elements = driver.find_elements(By.CLASS_NAME, 'item_tag.next')
score_area_elements = driver.find_elements(By.CLASS_NAME, 'score_area')
item_count_area_elements = driver.find_elements(By.CLASS_NAME, 'item_count_area.clear')

# 추출한 요소를 데이터프레임으로 만들기
data = {
    'tour_place': [element.text.strip() for element in tqdm(s_tit_elements, desc='Title')],
    'tour_place_theme': [element.text.strip() for element in tqdm(s_theme_elements, desc='Theme')],
    'tour_place_tag': [element.text.strip() for element in tqdm(item_tag_next_elements, desc='Item Tag')],
    'tour_place_score': [element.text.strip() for element in tqdm(score_area_elements, desc='Score Area')],
    'item_count_area': [element.text.strip() for element in tqdm(item_count_area_elements, desc='Item Count Area')]
}

tour_place_df = pd.DataFrame(data)

# 모든 값을 '없음'으로 인코딩
tour_place_df.replace("", "없음", inplace=True)
tour_place_df.replace(" ", "없음", inplace=True)

# 좋아요, 찜하기, 리뷰 값을 추출하여 데이터프레임에 추가
tour_place_df['like'] = tour_place_df['item_count_area'].apply(lambda x: x.split('\n')[1] if '좋아요' in x else '0')
tour_place_df['zzim'] = tour_place_df['item_count_area'].apply(lambda x: x.split('\n')[3] if '찜하기' in x else '0')
tour_place_df['review'] = tour_place_df['item_count_area'].apply(lambda x: x.split('\n')[5] if '리뷰' in x else '0')

# item_count_area 컬럼 삭제
tour_place_df.drop('item_count_area', axis=1, inplace=True)

# tour_place_tag가 '없음'인 행 삭제
tour_place_df = tour_place_df[tour_place_df['tour_place_tag'] != '없음']

# 별점 수치화
def convert_score(score):
    try:
        if '별점없음' in score:
            return 0
        else:
            score_value = score[9]
            return int(score_value) if score_value.isdigit() else 0
    except Exception as e:
        print(f"Error occurred with value: {score}. Error: {e}")
        return 0

tour_place_df['tour_place_score'] = tour_place_df['tour_place_score'].apply(convert_score)

# 데이터프레임 출력
tour_place_df

# WebDriver 종료
driver.quit()


아침 검색 1페이지입니다======


WebDriverException: Message: disconnected: received Inspector.detached event
  (failed to check if window was closed: disconnected: Unable to receive message from renderer)
  (Session info: chrome=120.0.6099.216)
Stacktrace:
	GetHandleVerifier [0x00007FF73DFB2142+3514994]
	(No symbol) [0x00007FF73DBD0CE2]
	(No symbol) [0x00007FF73DA776AA]
	(No symbol) [0x00007FF73DA61D04]
	(No symbol) [0x00007FF73DA61723]
	(No symbol) [0x00007FF73DA60746]
	(No symbol) [0x00007FF73DA606EB]
	(No symbol) [0x00007FF73DA5FF59]
	(No symbol) [0x00007FF73DA840AD]
	(No symbol) [0x00007FF73DB02AEF]
	(No symbol) [0x00007FF73DAE5D93]
	(No symbol) [0x00007FF73DAB4BDC]
	(No symbol) [0x00007FF73DAB5C64]
	GetHandleVerifier [0x00007FF73DFDE16B+3695259]
	GetHandleVerifier [0x00007FF73E036737+4057191]
	GetHandleVerifier [0x00007FF73E02E4E3+4023827]
	GetHandleVerifier [0x00007FF73DD004F9+689705]
	(No symbol) [0x00007FF73DBDC048]
	(No symbol) [0x00007FF73DBD8044]
	(No symbol) [0x00007FF73DBD81C9]
	(No symbol) [0x00007FF73D

In [12]:
tour_place_df

,tour_place,tour_place_theme,tour_place_tag,tour_place_score,like,zzim,review
0,성산일출봉(UNESCO 세계자연유산),서귀포시 > 성산,#부모 #자연경관 #포토스팟 #유네스코 #무장애관광 #공영관광지,5,364,"4,456",611
1,우도(해양도립공원),섬 속의 섬 > 우도,#맑음 #자연경관 #포토스팟 #어린이 #봄꽃 #유채꽃 #유네스코 #무장애관광 #공영관광지,5,230,"2,762",596
2,사려니숲길,제주시 > 조천,#커플 #흐림 #봄 #자연경관 #도보여행 #도보 #숲 #단풍 #공영관광지,5,250,"3,612",509
3,카멜리아힐,서귀포시 > 안덕,#맑음 #겨울 #힐링 #자연경관 #포토스팟 #어린이 #어트랙션 #동백 #수국 #무장...,5,195,"3,617",487
4,협재해수욕장,제주시 > 한림,#아이 #맑음 #여름 #자연경관 #체험 #레저/체험 #해변 #물놀이 #어린이 #수상...,5,191,"3,147",462
...,...,...,...,...,...,...,...
1482,황우럭만화천국사회적협동조합,제주시 > 한림,#어린이 #어트랙션,0,0,0,0
1483,효명사,서귀포시 > 남원,#사찰 #포토스팟 #어트랙션,0,2,14,0
1484,후포해변,제주시 > 제주시내,#포토스팟,0,1,5,0
1485,휘닉스 르쏠레이테라피,서귀포시 > 성산,#어트랙션,0,7,6,0


음식점

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from tqdm import tqdm
import time

# Chrome WebDriver 사용 (다운로드 링크: https://sites.google.com/chromium.org/driver/)
# driver_path = '/path/to/chromedriver'
driver = webdriver.Chrome()

url = "https://www.visitjeju.net/kr/detail/list?menuId=DOM_000001719000000000&cate1cd=cate0000000005#p1&pageSize=2000&sortListType=reviewcnt&viewType=map&isShowBtag&tag"
driver.get(url)

# 페이지 로드를 기다립니다. 여기서는 간단히 10초 대기하도록 설정하였습니다.
driver.implicitly_wait(15)

# 필요한 요소 추출
s_tit_elements = driver.find_elements(By.CLASS_NAME, 's_tit')
s_theme_elements = driver.find_elements(By.CLASS_NAME, 's_theme')
item_tag_next_elements = driver.find_elements(By.CLASS_NAME, 'item_tag.next')
score_area_elements = driver.find_elements(By.CLASS_NAME, 'score_area')
item_count_area_elements = driver.find_elements(By.CLASS_NAME, 'item_count_area.clear')

# 추출한 요소를 데이터프레임으로 만들기
data = {
    'tour_food': [element.text.strip() for element in tqdm(s_tit_elements, desc='Title')],
    'tour_food_theme': [element.text.strip() for element in tqdm(s_theme_elements, desc='Theme')],
    'tour_food_tag': [element.text.strip() for element in tqdm(item_tag_next_elements, desc='Item Tag')],
    'tour_food_score': [element.text.strip() for element in tqdm(score_area_elements, desc='Score Area')],
    'item_count_area': [element.text.strip() for element in tqdm(item_count_area_elements, desc='Item Count Area')]
}

tour_food_df = pd.DataFrame(data)

# 모든 값을 '없음'으로 인코딩
tour_food_df.replace("", "없음", inplace=True)
tour_food_df.replace(" ", "없음", inplace=True)

# 좋아요, 찜하기, 리뷰 값을 추출하여 데이터프레임에 추가
tour_food_df['like'] = tour_food_df['item_count_area'].apply(lambda x: x.split('\n')[1] if '좋아요' in x else '0')
tour_food_df['zzim'] = tour_food_df['item_count_area'].apply(lambda x: x.split('\n')[3] if '찜하기' in x else '0')
tour_food_df['review'] = tour_food_df['item_count_area'].apply(lambda x: x.split('\n')[5] if '리뷰' in x else '0')

# item_count_area 컬럼 삭제
tour_food_df.drop('item_count_area', axis=1, inplace=True)

# tour_place_tag가 '없음'인 행 삭제
tour_food_df = tour_food_df[tour_food_df['tour_food_tag'] != '없음']

# def convert_score(score):
#     try:
#         if '별점없음' in score:
#             return 0
#         else:
#             score_value = score[9]
#             return int(score_value) if score_value.isdigit() else 0
#     except Exception as e:
#         print(f"Error occurred with value: {score}. Error: {e}")
#         return 0

tour_food_df['tour_food_score'] = tour_food_df['tour_food_score'].apply(convert_score)

# 데이터프레임 출력
print(tour_food_df)

# WebDriver 종료
driver.quit()


Item Count Area: 100%|██████████| 1971/1971 [00:36<00:00, 54.02it/s]


           tour_food tour_food_theme  \
0               봄날카페        제주시 > 애월   
1     서연의집(영화 건축학개론)       서귀포시 > 남원   
2               명진전복        제주시 > 구좌   
3          몽상드애월(몽상)        제주시 > 애월   
4     제주김만복(만복이네김밥집)      제주시 > 제주시내   
...              ...             ...   
1965  흑돼지돈까스 뷔페 밀라네사      제주시 > 제주시내   
1967        흑본오겹 함덕점        제주시 > 조천   
1968           흑염소마루       서귀포시 > 남원   
1969            희신이네    서귀포시 > 서귀포시내   
1970              힌즈      제주시 > 제주시내   

                                          tour_food_tag  tour_food_score like  \
0     #음식 #콜드브루 #아메리카노 #바닐라라떼 #카페라떼 #아인슈페너 #카푸치노 #카페...                5   53   
1     #음식 #디저트 #빵 #와플 #에스프레소 #카페모카 #한류 #k-movie #드라마...                4   17   
2     #음식 #전복해물탕 #전복죽 #전복뚝배기 #전복버터구이 #전복물회 #전복돌솥밥 #전...                4   35   
3     #디저트 #음식 #아메리카노 #에스프레소 #카페모카 #카라멜마끼아또 #밀크티 #에이...                4   21   
4     #음식 #해물라면 #라면 #김밥 #주먹밥 #불갈비김밥 #오징어라면 #오징어무침 #비...                4   79   
...                              

In [14]:
tour_food_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1935 entries, 0 to 1970
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tour_food        1935 non-null   object
 1   tour_food_theme  1935 non-null   object
 2   tour_food_tag    1935 non-null   object
 3   tour_food_score  1935 non-null   int64 
 4   like             1935 non-null   object
 5   zzim             1935 non-null   object
 6   review           1935 non-null   object
dtypes: int64(1), object(6)
memory usage: 120.9+ KB


숙박

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from tqdm import tqdm
import time

# Chrome WebDriver 사용 (다운로드 링크: https://sites.google.com/chromium.org/driver/)
# driver_path = '/path/to/chromedriver'
driver = webdriver.Chrome()

url = "https://www.visitjeju.net/kr/detail/list?menuId=DOM_000001707000000000&cate1cd=cate0000000004#p1&pageSize=1000&sortListType=reviewcnt&viewType=map&isShowBtag&tag"
driver.get(url)

# 페이지 로드를 기다립니다. 여기서는 간단히 15초 대기하도록 설정하였습니다.
driver.implicitly_wait(15)

# 필요한 요소 추출
s_tit_elements = driver.find_elements(By.CLASS_NAME, 's_tit')
s_theme_elements = driver.find_elements(By.CLASS_NAME, 's_theme')
item_tag_next_elements = driver.find_elements(By.CLASS_NAME, 'item_tag.next')
score_area_elements = driver.find_elements(By.CLASS_NAME, 'score_area')
item_count_area_elements = driver.find_elements(By.CLASS_NAME, 'item_count_area.clear')

# 추출한 요소를 데이터프레임으로 만들기
data = {
    'tour_sleep': [element.text.strip() for element in tqdm(s_tit_elements, desc='Title')],
    'tour_sleep_theme': [element.text.strip() for element in tqdm(s_theme_elements, desc='Theme')],
    'tour_sleep_tag': [element.text.strip() for element in tqdm(item_tag_next_elements, desc='Item Tag')],
    'tour_sleep_score': [element.text.strip() for element in tqdm(score_area_elements, desc='Score Area')],
    'item_count_area': [element.text.strip() for element in tqdm(item_count_area_elements, desc='Item Count Area')]
}

tour_sleep_df = pd.DataFrame(data)

# 모든 값을 '없음'으로 인코딩
tour_sleep_df.replace("", "없음", inplace=True)
tour_sleep_df.replace(" ", "없음", inplace=True)

# 좋아요, 찜하기, 리뷰 값을 추출하여 데이터프레임에 추가
tour_sleep_df['like'] = tour_sleep_df['item_count_area'].apply(lambda x: x.split('\n')[1] if '좋아요' in x else '0')
tour_sleep_df['zzim'] = tour_sleep_df['item_count_area'].apply(lambda x: x.split('\n')[3] if '찜하기' in x else '0')
tour_sleep_df['review'] = tour_sleep_df['item_count_area'].apply(lambda x: x.split('\n')[5] if '리뷰' in x else '0')

# item_count_area 컬럼 삭제
tour_sleep_df.drop('item_count_area', axis=1, inplace=True)

# tour_place_tag가 '없음'인 행 삭제
tour_sleep_df = tour_sleep_df[tour_sleep_df['tour_sleep_tag'] != '없음']

# def convert_score(score):
#     try:
#         if '별점없음' in score:
#             return 0
#         else:
#             score_value = score[9]
#             return int(score_value) if score_value.isdigit() else 0
#     except Exception as e:
#         print(f"Error occurred with value: {score}. Error: {e}")
#         return 0

tour_sleep_df['tour_sleep_score'] = tour_sleep_df['tour_sleep_score'].apply(convert_score)

# 데이터프레임 출력
print(tour_sleep_df)

# WebDriver 종료
driver.quit()

Item Count Area: 100%|██████████| 988/988 [00:15<00:00, 62.04it/s]


                           tour_sleep tour_sleep_theme  \
0                           나미송 머무는 곳         제주시 > 애월   
1                          해비치호텔앤드리조트        서귀포시 > 표선   
2                              신라호텔제주        서귀포시 > 중문   
3                             롯데호텔 제주        서귀포시 > 중문   
4    휴양팬션 스위스마을 Swiss Village Pension         제주시 > 조천   
..                                ...              ...   
982                             환상의민박     섬 속의 섬 > 마라도   
983                              황재여관     섬 속의 섬 > 추차도   
984                             훼밀리민박        서귀포시 > 안덕   
985                            휴마루리조트     서귀포시 > 서귀포시내   
987                            힐링캠프펜션        서귀포시 > 남원   

                                        tour_sleep_tag  tour_sleep_score like  \
0                                      #정원 #주방기구 #자연경관                 5   26   
1    #숙소 #리조트 #가족호텔 #공공와이파이존 #양식레스토랑 #수영장 #주차장 #어린이...                 5   16   
2    #신라 #중문관광단지 #5성급호텔 #숙소 #게스트하우스 #호텔 #공공와이파이존 #양...      

In [16]:
tour_sleep_df

,tour_sleep,tour_sleep_theme,tour_sleep_tag,tour_sleep_score,like,zzim,review
0,나미송 머무는 곳,제주시 > 애월,#정원 #주방기구 #자연경관,5,26,32,175
1,해비치호텔앤드리조트,서귀포시 > 표선,#숙소 #리조트 #가족호텔 #공공와이파이존 #양식레스토랑 #수영장 #주차장 #어린이...,5,16,91,45
2,신라호텔제주,서귀포시 > 중문,#신라 #중문관광단지 #5성급호텔 #숙소 #게스트하우스 #호텔 #공공와이파이존 #양...,5,7,72,43
3,롯데호텔 제주,서귀포시 > 중문,#리조트 #숙소 #가족호텔 #수영장 #양식레스토랑 #자연경관 #공공와이파이존 #조식...,5,9,54,38
4,휴양팬션 스위스마을 Swiss Village Pension,제주시 > 조천,#마을관광 #올레길 #펜션,5,14,59,29
...,...,...,...,...,...,...,...
982,환상의민박,섬 속의 섬 > 마라도,#민박 #농어촌민박 #선착장 #힐링쉼터,0,0,0,0
983,황재여관,섬 속의 섬 > 추차도,#숙소 #포장판매 #여관,0,0,0,0
984,훼밀리민박,서귀포시 > 안덕,#농어촌민박 #펜션 #휴양펜션 #자연경관 #해변 #공공와이파이존 #해수욕장 #가족,0,1,0,0
985,휴마루리조트,서귀포시 > 서귀포시내,#휴양콘도 #공공와이파이존 #자연경관 #수영장 #환전,0,1,1,0


In [32]:
place_tags = tour_place_df['tour_place_tag'].str.split(' ', expand=True).stack()
food_tags = tour_food_df['tour_food_tag'].str.split(' ', expand=True).stack()
sleep_tags = tour_sleep_df['tour_sleep_tag'].str.split(' ', expand=True).stack()


place_tags_counts = place_tags.value_counts().head(10)
food_tags_counts = food_tags.value_counts().head(10)
sleep_tags_counts = sleep_tags.value_counts().head(10)

place_tags_counts,food_tags_counts,sleep_tags_counts


(#어트랙션        509
 #자연경관        406
 #            306
 #어린이         226
 #체험          225
 #맑음          220
 #반려동물동반입장    215
 #도보          208
 #도보여행        205
 #포토스팟        185
 Name: count, dtype: int64,
 #         1770
 #음식       1341
 #아메리카노     383
 #식당        358
 #카페라떼      330
 #에이드       308
 #라떼        270
 #차         270
 #바닐라라떼     253
 #에스프레소     209
 Name: count, dtype: int64,
 #공공와이파이존    492
 #주차장        316
 #바비큐        304
 #해수욕장       227
 #자연경관       211
 #조식         183
 #무장애관광      177
 #가족         168
 #독채         167
 #해변         158
 Name: count, dtype: int64)

In [33]:
# csv 파일로 저장
tour_place_df.to_csv('tour_place.csv', index=False)
tour_food_df.to_csv('tour_food.csv', index=False)
tour_sleep_df.to_csv('tour_sleep.csv', index=False)


: 